### Import

In [4]:
from config import pipeline_config
from etl_pipeline.pipeline import ETLPipeline
import config
from omegaconf import OmegaConf
from pyspark import Row
from config import columns_namespace
from etl_pipeline.data_processor_engine.spark_engine import SparkProcessingEngine
spark_engine = SparkProcessingEngine()

/env/ds/anaconda/envs/pipeline/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
Ivy Default Cache set to: /app/tutorial/dependencies/ivy2/cache
The jars for the packages stored in: /app/tutorial/dependencies/ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1091f4a0-045a-4e3f-bdbf-aa64156f3436;1.0
	confs: [default]


:: loading settings :: url = jar:file:/env/ds/anaconda/envs/pipeline/lib/python3.7/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
:: resolution report :: resolve 160ms :: artifacts dl 4ms
	:: modules in use:
	com.ibm.icu#icu4j;58.2 from central in [default]
	io.delta#delta-core_2.12;1.0.0 from central in [default]
	org.abego.treelayout#org.abego.treelayout.core;1.0.3 from central in [default]
	org.antlr#ST4;4.0.8 from central in [default]
	org.antlr#antlr-runtime;3.5.2 from central in [default]
	org.antlr#antlr4;4.7 from central in [default]
	org.antlr#antlr4-runtime;4.7 from central in [default]
	org.glassfish#javax.json;1.0.4 from central in [default]
	---------------------------------------------------------------------
	|             

### Pipeline

In [5]:
class MSPipeline(ETLPipeline):
    def convert_raw_to_standardized(self, df):
        return df

    def transform_standardized_to_cleansed(self, data):
        data = self.engine.set_ref_key(data)
        data = ms_customized_merge_df360_and_WM_Account_In_Scope(self.engine)
        data = self.engine.set_trigger_reasons(data)
        data = self.engine.merge_with_party_and_address_relationships(data)
        data = self.engine.set_beneficiary_hits(data)
        data = self.engine.set_clean_names(data)
        data = self.engine.set_concat_residue(data)
        data = self.engine.set_concat_address_no_change(data)
        data = self.engine.set_discovery_tokens(data)
        return data

    def transform_cleansed_to_application(self, cleansed_alert_df):
        agent_input_agg_df = self.engine.prepare_agent_inputs(cleansed_alert_df)
        return agent_input_agg_df


def ms_customized_merge_df360_and_WM_Account_In_Scope(engine):
    path = pipeline_config.WM_ACCOUNTS_IN_SCOPE_INPUT_PATH
    WM_Account_In_Scope = engine.spark_instance.read_csv(path)

    for col in WM_Account_In_Scope.columns:
        WM_Account_In_Scope = WM_Account_In_Scope.withColumnRenamed(col, col.replace(".", "_"))

    WM_Account_In_Scope.registerTempTable("WM_Account_In_Scope")

    df360_v2 = engine.spark_instance.spark_instance.sql(
        "select * from df360 left join WM_Account_In_Scope WM_Account "
        "on df360.SRC_SYS_ACCT_KEY = WM_Account.AD_SRC_SYS_ACCT_KEY"
    )
    return df360_v2


In [6]:
data_part1 = [
    Row(
        alert_id=1,
        SRC_REF_KEY='2019-05-09-12.07.37.400001',
        ACCT_NUM=1,
        PARTY_ID=51000001,
        ADDRESS_ID="A05002296231",
        TRIGGERED_BY=["John"],
        # AD_BNFL_NM=pattern.WL_NAME,
        CONCAT_ADDRESS="Jane Doe 111 A 11TH ST AAA 1A New Jersey NJ",
         WL_MATCHED_TOKENS="[\"John B Doe\"]",
        ADDRESS1_COUNTRY="",
        ADDRESS1_LINE1="test",
        ADDRESS1_LINE2="",
        ADDRESS1_LINE3="",
        ADDRESS1_LINE4="",
        ADDRESS1_LINE5="",
        ACCT_REGS_NM1="",

    ),
    Row(
        alert_id=2,
        SRC_REF_KEY='2020-01-23-08.29.18.794491',
        ACCT_NUM=2,
        PARTY_ID=51000002,
        ADDRESS_ID="A00150805711",
        TRIGGERED_BY=[columns_namespace.WL_ALIASES],
        # AD_BNFL_NM="AD_BNFL_NM",
        CONCAT_ADDRESS="XXX C/F YYY A ZET (DECD) Agent ZET (BENE) ABC DEF GHI 123 XYZ RD ZET XXX 11111",
        WL_MATCHED_TOKENS="[\"John B Doe\"]",
        ADDRESS1_COUNTRY="",
        ADDRESS1_LINE1="",
        ADDRESS1_LINE2="",
        ADDRESS1_LINE3="",
        ADDRESS1_LINE4="",
        ADDRESS1_LINE5="",
        ACCT_REGS_NM1="",
    )]

data_part2 = [
    Row(
        alert_id=1,
        STATUS_DESC="AASD",
        WL_NATIONALITY="[\"American\"]",
        ALERT_INTERNAL_ID="2",
        ENTITY_ID=1,
        ENTITY_VERSION=100,
        wl_name="test",

    ),
    Row(
        alert_id=2,
        STATUS_DESC="AASD",
        WL_NATIONALITY="[\"American\"]",
        ALERT_INTERNAL_ID="3",
        ENTITY_ID=2,
        ENTITY_VERSION=101,
        wl_name="test",

    )]

In [16]:
data_part_1 = spark_engine.spark_instance.spark_instance.createDataFrame(data_part1)
data_part_2 = spark_engine.spark_instance.spark_instance.createDataFrame(data_part2)
data = data_part_2.join(data_part_1, columns_namespace.ALERT_ID)

In [17]:
data.toPandas()

,alert_id,STATUS_DESC,WL_NATIONALITY,ALERT_INTERNAL_ID,ENTITY_ID,ENTITY_VERSION,wl_name,SRC_REF_KEY,ACCT_NUM,PARTY_ID,...,TRIGGERED_BY,CONCAT_ADDRESS,WL_MATCHED_TOKENS,ADDRESS1_COUNTRY,ADDRESS1_LINE1,ADDRESS1_LINE2,ADDRESS1_LINE3,ADDRESS1_LINE4,ADDRESS1_LINE5,ACCT_REGS_NM1
0,1,AASD,"[""American""]",2,1,100,test,2019-05-09-12.07.37.400001,1,51000001,...,[John],Jane Doe 111 A 11TH ST AAA 1A New Jersey NJ,"[""John B Doe""]",,test,,,,,
1,2,AASD,"[""American""]",3,2,101,test,2020-01-23-08.29.18.794491,2,51000002,...,[WL_ALIASES],XXX C/F YYY A ZET (DECD) Agent ZET (BENE) ABC ...,"[""John B Doe""]",,,,,,,


### Load config

In [18]:
conf = OmegaConf.load("config/config_app.yaml")
pipeline_config = conf.PIPELINE
columns_namespace = conf.SPARK_DATAFRAME_COLUMNS


In [19]:
pipeline_config

{'output-paths': 'output-paths', 'alerts-data': 'alerts-data', 'alerts-notes': 'alerts-notes', 'alerts-xml-data': 'alerts-xml-data', 'temp': '/tmp', 'sanctions-360': 'sanctions-360', 'input-record-columns': 'input-record-columns', 'id-column-name': 'id-column-name', 'WM_ACCOUNTS_IN_SCOPE_INPUT_PATH': '../ms-name-screening-poc/test-data/Sanctions/WM/WM_Account_In_Scope.csv', 'WM_PARTIES_IN_SCOPE_INPUT_PATH': '../ms-name-screening-poc/test-data/Sanctions/WM/WM_Parties_In_Scope.csv', 'df360-input-path': 'df360-input-path', 'df360-output-path': 'df360-output-path', 'fuzziness': 100, 'PARTY_AND_ADDRESS_RELATIONSHIP_INPUT_PATH': '../tests/shared/Party_And_Address_Relationship.csv', 'decision-tree': 'decision-tree', 'alerts': 'alerts', 'alert-type': 'alert-type', 'WM_ADDRESS_COLS_TO_CONVERT_FROM_JSON': [], 'ISG_ACCOUNT_COLS_TO_CONVERT_FROM_JSON': [], 'AP_COLUMNS': ['ADDRESS1_COUNTRY', 'ADDRESS1_LINE1', 'ADDRESS1_LINE2', 'ADDRESS1_LINE3', 'ADDRESS1_LINE4', 'ADDRESS1_LINE5', 'CONCAT_ADDRESS', '

# Pipeline example

In [20]:
class MSPipeline(ETLPipeline):
    def convert_raw_to_standardized(self, df):
        return df

    def transform_standardized_to_cleansed(self, data):
        data = self.engine.set_ref_key(data)
        data.registerTempTable("df360")
        data = self.ms_customized_merge_df360_and_WM_Account_In_Scope()
        data = self.engine.set_trigger_reasons(data)
        data = self.engine.merge_with_party_and_address_relationships(data)
        data = self.engine.set_beneficiary_hits(data)
        data = self.engine.set_clean_names(data)
        data = self.engine.set_concat_residue(data)
        data = self.engine.set_concat_address_no_change(data)
        data = self.engine.set_discovery_tokens(data)
        return data

    def transform_cleansed_to_application(self, cleansed_alert_df):
        agent_input_agg_df = self.engine.prepare_agent_inputs(cleansed_alert_df)
        return agent_input_agg_df

    def ms_customized_merge_df360_and_WM_Account_In_Scope(self):
        path = self.pipeline_config.WM_ACCOUNTS_IN_SCOPE_INPUT_PATH
        WM_Account_In_Scope = self.engine.spark_instance.read_csv(path)

        for col in WM_Account_In_Scope.columns:
            WM_Account_In_Scope = WM_Account_In_Scope.withColumnRenamed(col, col.replace(".", "_"))

        WM_Account_In_Scope.registerTempTable("WM_Account_In_Scope")

        df360_v2 = self.engine.spark_instance.spark_instance.sql(
            "select * from df360 left join WM_Account_In_Scope WM_Account "
            "on df360.SRC_SYS_ACCT_KEY = WM_Account.AD_SRC_SYS_ACCT_KEY"
        )
        return df360_v2


In [21]:
uut = MSPipeline(spark_engine, config=pipeline_config)
result = uut.transform_standardized_to_cleansed(data)

In [22]:
result = uut.transform_cleansed_to_application(result)

In [25]:
pd_result = result.toPandas()

/env/ds/anaconda/envs/pipeline/lib/python3.7/site-packages/pyspark/sql/pandas/conversion.py:186: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series


# Prepare agent input

In [3]:
syntax = "proto3";

package silenteight.datasource.api.date.v1;

import "google/api/annotations.proto";

option java_package = "com.silenteight.datasource.api.date.v1";
option java_outer_classname = "DateProto";
option java_multiple_files = true;

service DateInputService {
  rpc BatchGetMatchDateInputs (BatchGetMatchDateInputsRequest) returns (stream BatchGetMatchDateInputsResponse) {
    option (google.api.http) = {
      post: "/agent-inputs/date/versions/v1/matches:batchGet"
      body: "*"
    };
  }
}

message BatchGetMatchDateInputsRequest {
  // The list of matches data is requested for, in form of `alerts/<Alert ID>/matches/<Match ID>`.
  repeated string matches = 1;
  // The list of features the input data for each match is requested, in form of `features/<Feature ID>`.
  repeated string features = 2;
}

message BatchGetMatchDateInputsResponse {
  repeated DateInput date_inputs = 1;
}

message DateInput {
  string match = 1;
  repeated DateFeatureInput date_feature_inputs = 2;
}

message DateFeatureInput {
  string feature = 1;
  repeated string alerted_party_dates = 2;
  repeated string watchlist_dates = 3;

  EntityType alerted_party_type = 4;
  SeverityMode mode = 5;

  enum EntityType {
    ENTITY_TYPE_UNSPECIFIED = 0;
    INDIVIDUAL = 1;
    ORGANIZATION = 2;
  }

  enum SeverityMode {
    NORMAL = 0;
    STRICT = 1;
  }
}


SyntaxError: invalid syntax (3968094792.py, line 3)

In [ ]:
message DateFeatureInput {
  string feature = 1;
  repeated string alerted_party_dates = 2;
  repeated string watchlist_dates = 3;

  EntityType alerted_party_type = 4;
  SeverityMode mode = 5;

  enum EntityType {
    ENTITY_TYPE_UNSPECIFIED = 0;
    INDIVIDUAL = 1;
    ORGANIZATION = 2;
  }

  enum SeverityMode {
    NORMAL = 0;
    STRICT = 1;
  }
}


In [35]:
entity_type = [column for column in pd_result.columns if "alerted_party_type" in column]

In [43]:
[column for column in pd_result.columns if "severity" in column.lower()]

[]

In [41]:
pd_result[[column for column in pd_result.columns if "ALL_PARTY_TYPES" in column]]

,ALL_PARTY_TYPES
0,"[Individual, Individual]"
1,[Individual]


In [28]:
dob_columns = [column for column in pd_result.columns if "dob" in column]

In [50]:
item = pd_result[dob_columns].values

In [51]:
item

array([[list(['01/02/1972', '01/02/1938']), None,
        list(['01/02/1972', '01/02/1938']), list([])],
       [list(['01/02/1960']), None, list(['01/02/1960']), list([])]],
      dtype=object)

In [59]:
from silenteight.datasource.api.date.v1.date_pb2 import DateFeatureInput
from silenteight.datasource.agentinput.api.v1.agent_input_pb2 import AgentInput, FeatureInput


In [42]:
? DateFeatureInput

Init signature:  DateFeatureInput(self, /, *args, **kwargs)
Docstring:      A ProtocolMessage
File:           /env/ds/anaconda/envs/pipeline/lib/python3.7/site-packages/silenteight/datasource/api/date/v1/date_pb2.py
Type:           GeneratedProtocolMessageType
Subclasses:     


In [61]:
message = DateFeatureInput(feature="dob", alerted_party_dates=item[0][0], watchlist_dates=item[0][1], alerted_party_type=1, mode=2)

In [72]:
from google.protobuf.any_pb2 import Any
target =  Any()


In [73]:
FeatureInput(feature=message.feature, agent_feature_input=target.Pack((message)))

feature: "dob"